## 탐색적 데이터 분석
    1. 종족 카테고리화 ( 필요 한가?? )
    2. controlgroup 묶기
    3. 특정 컬럼 비율화 
        - controlgroup
        - attack
        - worker

#### 라이브러리 import

In [ ]:
import pandas as pd 
import pickle 
import numpy as np
import re
from tqdm import tqdm
from tqdm import trange
pd.set_option( 'display.max_rows' , 2000)
pd.set_option( 'display.max_columns' , 200)



### 0. 데이터 description

In [ ]:
data = pd.read_csv('./data/For_Analysis_0318.csv' , index_col = 'game_id')

columns = ['winner', 'p0_species', 'p1_species', 'playtime', 'p0_Camera',
       'p0_Selection', 'p0_Ability', 'p0_Right Click', 'p0_SetControlGroup',
       'p0_GetControlGroup', 'p0_AddToControlGroup', 'p0_ControlGroup',
       'p1_Camera', 'p1_Selection', 'p1_Ability', 'p1_Right Click',
       'p1_SetControlGroup', 'p1_GetControlGroup', 'p1_AddToControlGroup',
       'p1_ControlGroup', 'rate_Camera', 'rate_Selection', 'rate_Ability',
       'rate_Right Click', 'rate_SetControlGroup', 'rate_GetControlGroup',
       'rate_AddToControlGroup', 'rate_ControlGroup', 'workers_p0',
       'workers_p1', 'p1_attack', 'total_attack', 'p0_ControlGroup_action',
       'p1_ControlGroup_action', 'rate_worker']
       
total_attack              0.002201
rate_ControlGroup         0.002256
p1_ControlGroup           0.003303
p0_species                0.004561
p1_species                0.004585
playtime                  0.004650
p1_Camera                 0.007286
rate_Camera               0.007579
p0_Camera                 0.009406
p0_ControlGroup           0.009969
p1_AddToControlGroup      0.032163
p0_Right Click            0.032989
p0_AddToControlGroup      0.033050
p1_Selection              0.040425
p0_Selection              0.040724
rate_AddToControlGroup    0.040801
p1_Right Click            0.041839
p1_attack                 0.044019
p0_SetControlGroup        0.049926
p1_SetControlGroup        0.051379
p1_GetControlGroup        0.054063
p0_Ability                0.059533
p1_Ability                0.060784
p0_GetControlGroup        0.068521
workers_p0                0.076637
workers_p1                0.079426
rate_SetControlGroup      0.079886
rate_Selection            0.093696
rate_Right Click          0.096837
rate_GetControlGroup      0.145908
rate_Ability              0.201047
winner                    1.000000

### 1. 종족 수치화

    T --> 0 , Z --> 1 , P --> 2

In [ ]:
def species_to_number (value):
    if value == 'T':
        return 0
    elif value == 'Z':
        return 1
    elif value == 'P':
        return 2
data['p0_species'] = data['p0_species'].apply(species_to_number)
data['p1_species'] = data['p1_species'].apply(species_to_number)

### 2. controlgroup 합치기 
    
    - 모두 합치기
    - 비율 뽑기

In [ ]:
# 모두 합치기 , 비율뽑기 (control group action )

player = ['p0_' , 'p1_']
for p in player:
    data[p+'ControlGroup_action'] =  data[p+'SetControlGroup'] + data[p+'GetControlGroup'] + data[p+'AddToControlGroup']+ data[p+'ControlGroup']


### 3. 특정 컬럼 비율화
    - controlGroup
    - attack 
    - worker

In [ ]:
# ControlGroup 비율화 
data['rate_ControlGroup'] = data['p1_ControlGroup_action'] / (data['p0_ControlGroup_action'] + data['p1_ControlGroup_action'])
# attack 비율화
data['rate_attack'] = data['p1_attack'] / ( data['p1_attack'] + data['p0_attack'])
# worker 비율화 
data['rate_worker'] = data['workers_p1'] / ( data['workers_p1'] + data['workers_p0'])

### 4. 종족별 행동 컬럼화
1. 종족별 행동 dict 만들기 
    - build, unit , upgrade
2. raw_data action_code 칼럼 만들기
3. counts 하기

1. 종족별 행동 dict 만들기

In [ ]:
def make_new_dic(dic):
    new_dic = {}
    for species in dic:
        for action in dic[species]:
            for code in dic[species][action]:
                new_dic.update( {code+species : action })
    return new_dic

2. raw_data action_code 만들기

In [ ]:
def get_action_code(df):
    code_reg = re.compile('[0-9A-Z]{3}[0-9A-Z]?')
    data = df['event_contents']
    species = df['species']
    try:
        code = code_reg.findall(data)[0]
        return code + species
    except:
        return np.nan

def get_action( data ):
    global act_list
    try:
        return act_list[data]
    except:
        return np.nan
    
    
    
# raw_data 받아서 , action_code , action 컬럼 붙여주기

def make_action_code_columns( data ):
    data = data.loc[ data['event'] == 'Ability']
    data['action_code'] = data.apply(get_action_code  ,axis = 1)
    data['action'] = data['action_code'].apply(get_action)
    return data


3. count 하기
    - 새로운 컬럼 생성 ( index = game_id , columns = p1_unit , p1_build ,,,,, )

In [ ]:
def Make_ActCounts_Columns( data ):

    input_data = make_action_code_columns( data )
    
    tmp = input_data.set_index('game_id')
    idx_ = sorted(list(set(tmp.index)))


    p1_unit_col = []
    p1_build_col = []
    p1_upgrade_col = []
    p0_unit_col = []
    p0_build_col = []
    p0_upgrade_col = []
    g_id = []
    
    print( 'counting Now!!')

    for i in tqdm(idx_[:]):
        one_game =tmp.loc[i]
        one_game.dropna(axis = 0 , inplace = True)
        try:
            p1_game = one_game.loc[ one_game['player'] == 1]

            p1_unit = len(p1_game.loc[ p1_game['action'] == 'unit'])
            p1_build = len(p1_game.loc[ p1_game['action'] == 'build'])
            p1_upgrade = len(p1_game.loc[ p1_game['action'] == 'upgrade'])

            p1_unit_col.append(p1_unit)
            p1_build_col.append(p1_build)
            p1_upgrade_col.append(p1_upgrade)
        except:
            p1_unit_col.append(0)
            p1_build_col.append(0)
            p1_upgrade_col.append(0)

        try:    
            p0_game = one_game.loc[ one_game['player'] == 0]

            p0_unit = len(p0_game.loc[ p0_game['action'] == 'unit'])
            p0_build = len(p0_game.loc[ p0_game['action'] == 'build'])
            p0_upgrade = len(p0_game.loc[ p0_game['action'] == 'upgrade'])


            p0_unit_col.append(p0_unit)
            p0_build_col.append(p0_build)
            p0_upgrade_col.append(p0_upgrade)

        except:
            p0_unit_col.append(0)
            p0_build_col.append(0)
            p0_upgrade_col.append(0)

        g_id.append(i)

    res = pd.DataFrame({'game_id': g_id , 'p1_unit':p1_unit_col ,'p1_build' : p1_build_col,'p1_upgrade' : p1_upgrade_col,'p0_unit': p0_unit_col ,'p0_build': p0_build_col,'p0_upgrade': p0_upgrade_col})
    return res.set_index('game_id' , drop = True )



##### 뽑아 봐, 라

In [ ]:
raw_data = pd.read_csv('data/test.csv')

with open( 'data/act_list.p' , 'rb' ) as file: 
    act_list = pickle.load( file )


act_list = make_new_dic(act_list)
col_actcounts = Make_ActCounts_Columns( raw_data )

D:\anaconda\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


counting Now!!


  0%|          | 0/16785 [00:00<?, ?it/s]D:\anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  6%|▋         | 1068/16785 [00:04<01:13, 213.53it/s]D:\anaconda\lib\site-packages\pandas\core\series.py:4784: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(result)
100%|██████████| 16785/16785 [01:16<00:00, 220.27it/s]


### +++상관관계 , 컬럼 selection

In [ ]:
idx__=list(set(col_actcounts.index)^set(data.index))

In [ ]:
idx__

[41795, 46435]

In [ ]:
col_actcounts.loc[41795]=data.loc[41795].fillna(0.5) # rate feature nan fillna 0? 0.5? 0.5가 중립값아님?
col_actcounts.loc[46435]=data.loc[46435].fillna(0.5)

In [ ]:
col_actcounts

,p1_unit,p1_build,p1_upgrade,p0_unit,p0_build,p0_upgrade
game_id,,,,,,
38872,16.0,14.0,1.0,13.0,8.0,1.0
38873,17.0,11.0,0.0,12.0,6.0,1.0
38874,6.0,6.0,1.0,12.0,7.0,1.0
38875,22.0,11.0,2.0,29.0,18.0,4.0
38876,1.0,3.0,1.0,0.0,2.0,0.0
...,...,...,...,...,...,...
55656,9.0,19.0,0.0,8.0,4.0,0.0
55657,2.0,3.0,0.0,4.0,2.0,0.0
55658,12.0,7.0,0.0,5.0,2.0,0.0


In [ ]:
data.shape, col_actcounts.shape

((16787, 36), (16787, 6))

In [ ]:
con_data = pd.merge( data, col_actcounts , left_index=True , right_index=True , how = 'inner')
aa =['unit' , 'build' , 'upgrade']
for n in tqdm(aa):
    con_data['rate_'+n ] = con_data['p1_'+n] /  con_data['p0_'+n] + con_data['p1_'+n]
   
 #abs(con_data.corr()['winner']).sort_values()


100%|██████████| 3/3 [00:00<00:00, 1002.54it/s]


In [ ]:
con_data

,p0_species,p1_species,playtime,p0_Camera,p0_Selection,p0_Ability,p0_AddToControlGroup,p0_SetControlGroup,p0_Right Click,p0_GetControlGroup,p0_ControlGroup,p1_Camera,p1_Selection,p1_Ability,p1_AddToControlGroup,p1_SetControlGroup,p1_Right Click,p1_GetControlGroup,p1_ControlGroup,rate_Camera,rate_Selection,rate_Ability,rate_AddToControlGroup,rate_SetControlGroup,rate_Right Click,rate_GetControlGroup,rate_ControlGroup,workers_p0,workers_p1,p0_attack,p1_attack,total_attack,p0_ControlGroup_action,p1_ControlGroup_action,rate_attack,rate_worker,p1_unit,p1_build,p1_upgrade,p0_unit,p0_build,p0_upgrade,rate_unit,rate_build,rate_upgrade
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38872,2,2,7.06,232,52,31,1,5,241,9,0,467,41,38,4,0,122,43,0,0.668097,0.440860,0.550725,0.800000,0.000000,0.336088,0.826923,0.758065,11,12,5,2,7,15,47,0.285714,0.521739,16.0,14.0,1.0,13.0,8.0,1.0,17.230769,15.750000,2.00
38873,1,2,6.07,200,114,34,0,0,51,18,4,146,19,28,1,5,68,122,0,0.421965,0.142857,0.451613,1.000000,1.000000,0.571429,0.871429,0.853333,7,13,0,0,0,22,128,NaN,0.650000,17.0,11.0,0.0,12.0,6.0,1.0,18.416667,12.833333,0.00
38874,0,0,4.41,245,42,33,0,7,79,63,0,220,36,29,0,5,108,426,0,0.473118,0.461538,0.467742,0.000000,0.416667,0.577540,0.871166,0.860279,9,5,9,4,13,70,431,0.307692,0.357143,6.0,6.0,1.0,12.0,7.0,1.0,6.500000,6.857143,2.00
38875,0,2,7.58,515,123,95,8,9,218,255,0,357,99,58,1,2,322,24,0,0.409404,0.445946,0.379085,0.111111,0.181818,0.596296,0.086022,0.090301,21,19,13,17,30,272,27,0.566667,0.475000,22.0,11.0,2.0,29.0,18.0,4.0,22.758621,11.611111,2.50
38876,1,1,2.50,129,25,7,2,2,30,387,0,161,35,8,0,1,7,1,0,0.555172,0.583333,0.533333,0.000000,0.333333,0.189189,0.002577,0.005089,4,2,0,0,0,391,2,NaN,0.333333,1.0,3.0,1.0,0.0,2.0,0.0,inf,4.500000,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55654,1,0,6.14,258,186,37,2,6,136,693,0,318,108,53,15,14,149,515,0,0.552083,0.367347,0.588889,0.882353,0.700000,0.522807,0.426325,0.436948,7,19,9,8,17,701,544,0.470588,0.730769,22.0,8.0,2.0,6.0,6.0,1.0,25.666667,9.333333,4.00
55655,1,0,9.58,810,317,62,0,7,158,217,0,638,103,75,2,0,174,50,0,0.440608,0.245238,0.547445,1.000000,0.000000,0.524096,0.187266,0.188406,11,10,8,18,26,224,52,0.692308,0.476190,21.0,16.0,5.0,11.0,10.0,4.0,22.909091,17.600000,6.25
55656,2,2,6.24,330,67,14,0,1,106,0,0,328,52,36,0,0,251,28,2,0.498480,0.436975,0.720000,0.000000,0.000000,0.703081,1.000000,0.967742,6,8,0,6,6,1,30,1.000000,0.571429,9.0,19.0,0.0,8.0,4.0,0.0,10.125000,23.750000,NaN


In [ ]:
con_data.columns

Index(['p0_species', 'p1_species', 'playtime', 'p0_Camera', 'p0_Selection',
       'p0_Ability', 'p0_AddToControlGroup', 'p0_SetControlGroup',
       'p0_Right Click', 'p0_GetControlGroup', 'p0_ControlGroup', 'p1_Camera',
       'p1_Selection', 'p1_Ability', 'p1_AddToControlGroup',
       'p1_SetControlGroup', 'p1_Right Click', 'p1_GetControlGroup',
       'p1_ControlGroup', 'rate_Camera', 'rate_Selection', 'rate_Ability',
       'rate_AddToControlGroup', 'rate_SetControlGroup', 'rate_Right Click',
       'rate_GetControlGroup', 'rate_ControlGroup', 'workers_p0', 'workers_p1',
       'p0_attack', 'p1_attack', 'total_attack', 'p0_ControlGroup_action',
       'p1_ControlGroup_action', 'rate_attack', 'rate_worker', 'p1_unit',
       'p1_build', 'p1_upgrade', 'p0_unit', 'p0_build', 'p0_upgrade',
       'rate_unit', 'rate_build', 'rate_upgrade'],
      dtype='object')

In [ ]:
selected_columns = ['playtime','p0_species', 'p1_species','rate_Selection', 
                    'rate_Ability', 'rate_Right Click',
                   'rate_SetControlGroup', 'rate_GetControlGroup',
                   'rate_AddToControlGroup', 'rate_ControlGroup', 'rate_attack',
                   'p0_ControlGroup_action', 'p1_ControlGroup_action','rate_worker',
                   'rate_unit', 'rate_build', 'rate_upgrade',] #'winner'

data_selected = con_data[selected_columns]
data_selected.shape

(16787, 17)

In [ ]:
data_selected.to_csv('./data/Test_0318.csv')